In [ ]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("data.tsv", index_col = False, sep="\\t")
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0,0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0,0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0,0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31
3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5,0,0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31
4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5,0,0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042230,US,52895472,RVR5HEPUCU2C0,B00005RE3U,587330987,"Sterling Silver Toggle Bracelet, Round Charm",Jewelry,4,0,0,N,N,nice bracelet,It is nice looking and everything (it is sterl...,2001-11-25
3042231,US,40225563,R7FC4HLPYQCBV,B00005OTBA,945322302,10K Yellow Gold &#46;01 ct&#46; Diamond Heart ...,Jewelry,4,6,7,N,N,beautiful heart for everyday wear,"my boyfriend bought me this last christmas, an...",2001-11-24
3042232,US,51016122,R1UD30PTI793U2,0930948246,698292720,Country & Blues Harmonica for the Absolute Beg...,Jewelry,4,9,9,N,N,Great for the Absolute Beginner,This is a great way to quickly start learning ...,2001-11-24
3042233,US,43629114,R1DXZZ4QSXZLTP,B00005OTAJ,274507819,14K Gold Diamond Cut Etched Hoop Earrings,Jewelry,5,8,10,N,N,Dazzling,the 14kt gold earrings look remarkable...would...,2001-11-18


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


## 1. Dataset Generation

In [ ]:
df = df[['review_body','star_rating']]
df = df.dropna()
df.isnull().sum()

review_body    0
star_rating    0
dtype: int64

In [ ]:
def select_data(df):
    data = df[df['star_rating'] == '1'].sample(n=20000) # select 20000 first calss data that rating is 1
    for i in range(2,6):
        data1 = df[df['star_rating']== str(i)].sample(n=20000) # select data of class 2,3,4,5 by looping
        data = data.append(data1, ignore_index=True)
    return data
  
data =select_data(df)
data['star_rating'] = data['star_rating'].apply(int) #change star_rating data to int type
data

,review_body,star_rating
0,Disappointed that this was sent out when you c...,1
1,"I returned them! They are really small, you ev...",1
2,FELL APART BEFORE I EVEN WORE IT!!! SO FRAGILE...,1
3,When I presented this charm I was informed tha...,1
4,Deceiving because they don't tell u the earrin...,1
...,...,...
99995,This ring is super pretty! The size I ordered ...,5
99996,The stones in this are larger than they appear...,5
99997,Well made.,5
99998,Just what I needed.,5


In [ ]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import contractions
def word_lower(data):
    data['review_body'] = data['review_body'].str.lower() # convert the all reviews data into the lower case
    return data

def remove_HTML_URL(text):
    text = re.sub('<.*?>', ' ', text)
    text = re.sub("https://\S+|www\.\S+", ' ', text) ## remove all html/url symbol 
    return text

def remove_non_alphabetical(text):
    text = re.sub(r'[^A-Za-z0-9 ]+', '',text) # remove all non-alphabetical character
    return text

def remove_extra_spaces(text):
    res = " ".join(text.split()) # remove extra space
    return res

def perform_contractions(text): # perform contractions on the reviews
    list1 = []
    for word in text.split():
        list1.append(contractions.fix(word))
    return " ".join(list1)

data = word_lower(data)
data['review_body'] = data['review_body'].apply(lambda x: perform_contractions(x))
data['review_body'] = data['review_body'].apply(lambda x: remove_HTML_URL(x))
data['review_body'] = data['review_body'].apply(lambda x: remove_non_alphabetical(x))
data['review_body'] = data['review_body'].apply(lambda x: remove_extra_spaces(x))

I did the basic data clean for the reivew data in order to decrease bad effect of the special character to the model

In [ ]:
from sklearn.model_selection import train_test_split
X = data['review_body']
y = data['star_rating']
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(X, y,test_size=0.2, random_state=42)
x_train_data

75220    the shipping took longer than expected but i s...
48955    it is really cute and looks better than the pi...
44966    the necklace pretty nice with pendant design b...
13568    do not waste your money on this i was astonish...
92727    i bought this as a friendship ring for a frien...
                               ...                        
6265     the idea and the actual content was nice but t...
54886                     picture was nicer than in person
76820    i chose this because i like two toned jewelry ...
860      i bought this ring to use while at work and af...
15795                        it the balls were way too big
Name: review_body, Length: 80000, dtype: object

## 2. Word Embedding

In [ ]:
import gensim.downloader as api
google_model = api.load("word2vec-google-news-300") #Load the pretrained “word2vec-google-news-300” Word2Vec model

In [ ]:
google_model.wv.similarity(w1 = 'nice', w2 = 'great')

0.64546573

In [ ]:
google_model.wv.similarity(w1 = 'picture', w2 = 'photo')

0.5958257

In [ ]:
google_model.wv.similarity(w1 = 'watch', w2 = 'look')

0.33878717

## (b)

In [ ]:
x_train_data = x_train_data.to_list()
x_train_data_list = []
for i in x_train_data:
  list1 = []
  for j in i.split():
    list1.append(j)
  x_train_data_list.append(list1)

In [ ]:
from gensim.models import Word2Vec

Word2Vec_model = Word2Vec(sentences = x_train_data_list, size = 300, window = 11, min_count=10) # create the word2vec model with 300 embedding size, 11 window size, and min count 10 word

In [ ]:
Word2Vec_model.wv.similarity(w1 = 'nice', w2 = 'great')

0.67641896

In [ ]:
Word2Vec_model.wv.similarity(w1 = 'picture', w2 = 'photo')

0.93262094

In [ ]:
Word2Vec_model.wv.similarity(w1 = 'watch', w2 = 'look')

-0.0027710078

Pretrained word2vec-google-news-300 model: cosine similarity of word nice and great is 0.64546573, cosine similarity of word picture and photo is 0.5958257, cosine similarity of word watch and look 0.33878717.
Myself Word2Vec model: cosine similarity of word nice and great is 0.67641896, cosine similarity of word picture and photo is 0.93262094, cosine similarity of word watch and look -0.0027710078.
For the fist example, the result of two model are similar; for the second example, the result of myself Word2Vec model is much better than pretrained model; for the third example, the result of pretrained model is much better than myself Word2Vec model

### 3.Sample Model

In [ ]:
x_train_vector_list = [] ## list to keep all vector of each word of train data after use word2vec model
for i in range(len(x_train_data_list)):
  list1 = np.zeros(300)
  for word in x_train_data_list[i]: # iterate every word of a review
    if word in google_model.wv.vocab: # check if word is in the model's vocabulary
      list1 += google_model.wv[word] # sum all the vector 
    list1 = list1/len(x_train_data_list[i]) # count the averge vector of a review data
  x_train_vector_list.append(list1)

In [ ]:
x_train_vector_list = np.array(x_train_vector_list) # convert the list to array
x_train_vector_list.shape

(80000, 300)

In [ ]:
x_test_data = x_test_data.to_list() # convert test data to list
x_test_data_list = []
for i in x_test_data:
  list1 = []
  for j in i.split():
    list1.append(j) # append every word of review
  x_test_data_list.append(list1)

In [ ]:
x_test_vector_list = [] ## list to keep all vector of each word of test data after use word2vec model
for i in range(len(x_test_data_list)):
  list1 = np.zeros(300)
  for word in x_test_data_list[i]:
    if word in google_model.wv.vocab:
      list1 += google_model.wv[word]
    list1 = list1/len(x_test_data_list[i])
  x_test_vector_list.append(list1)

In [ ]:
x_test_vector_list = np.array(x_test_vector_list) # convert list to array
x_test_vector_list.shape

(20000, 300)

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

Perceptron_model = Perceptron() 
Perceptron_model.fit(x_train_vector_list,y_train_data) # train the perceptron model
y_predict_data = Perceptron_model.predict(x_test_vector_list) # predict the test data

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4']
summary = classification_report(y_test_data, y_predict_data, target_names=target_names, output_dict = True)
summary['accuracy']

0.2591

For the last homework, the accurary of Perceptron is 0.39035, and the accurary of Perceptron of this homework is 0.2591. Compare the result, the accuracy of TF-IDF feature is much better than the accuracy of pretrained Word2Vec features. 

In [ ]:
from sklearn.svm import SVC

SVM = SVC(kernel='linear', gamma='auto')
SVM.fit(x_train_vector_list,y_train_data) # train the perceptron model
SVM_y_predict_data = SVM.predict(x_test_vector_list) #predict the test data

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4']
summary_SVM = classification_report(y_test_data, SVM_y_predict_data, target_names=target_names, output_dict = True)
summary_SVM['accuracy']

0.2578

For the last homework, the accurary of SVM is 0.5036, and the accurary of SVM of this homework is 0.2578. Compare the result, the accuracy of TF-IDF feature is much better than the accuracy of pretrained Word2Vec features.

## 4. Feedforward Neural Networks

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

MLP = Sequential() # ceate MLP model
MLP.add(Dense(300,input_shape=(300,), activation='relu')) # input layer with 300 node 
MLP.add(Dense(50, activation='relu')) # first hidden layer with 50 node
MLP.add(Dense(10, activation='relu')) # second hidden layer with 10 node
MLP.add(Dense(5, activation='sigmoid')) # output layer with 5 node beacuse there are 5 class


In [ ]:
import tensorflow as tf
x_train_tensor = tf.convert_to_tensor(x_train_vector_list) # convert x train data to tensor type
x_test_tensor = tf.convert_to_tensor(x_test_vector_list) # convert x test data to tensor type

In [ ]:
from sklearn.preprocessing import OneHotEncoder

OHE_model = OneHotEncoder(handle_unknown='ignore') # apply one hot encoding
OHE_y_train_data = OHE_model.fit_transform(np.array(y_train_data).reshape(-1,1)).toarray() # convert y train data to one hot coding
OHE_y_train_data


array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [ ]:
MLP.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) # compile the model

MLP.fit(x_train_tensor, OHE_y_train_data, epochs=50, batch_size=500) # train MLP model

Epoch 1/50
160/160 [==============================] - 3s 3ms/step - loss: 1.5595 - accuracy: 0.2722
Epoch 2/50
160/160 [==============================] - 0s 3ms/step - loss: 1.5113 - accuracy: 0.3076
Epoch 3/50
160/160 [==============================] - 0s 3ms/step - loss: 1.4926 - accuracy: 0.3195
Epoch 4/50
160/160 [==============================] - 0s 3ms/step - loss: 1.4819 - accuracy: 0.3266
Epoch 5/50
160/160 [==============================] - 1s 3ms/step - loss: 1.4730 - accuracy: 0.3319
Epoch 6/50
160/160 [==============================] - 0s 3ms/step - loss: 1.4654 - accuracy: 0.3359
Epoch 7/50
160/160 [==============================] - 1s 3ms/step - loss: 1.4590 - accuracy: 0.3386
Epoch 8/50
160/160 [==============================] - 0s 3ms/step - loss: 1.4531 - accuracy: 0.3415
Epoch 9/50
160/160 [==============================] - 0s 3ms/step - loss: 1.4486 - accuracy: 0.3455
Epoch 10/50
160/160 [==============================] - 0s 3ms/step - loss: 1.4456 - accuracy: 0.3469

In [ ]:
from tensorflow.python.ops.math_ops import arg_max
y_pre = MLP.predict(x_test_tensor) # predit y test data
y_pre_list = []
for item in y_pre:
  y_pre_list.append(np.argmax(item,axis=0)+1) # select the class of predict

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_data, y_pre_list) # count the accuracy
accuracy

0.33795

### (b)

In [ ]:
x_train_10vector_list = [] # create a list to keep the first 10 word of each review of train data
x_train_data_list1 = x_train_data_list.copy()
for i in range(len(x_train_data_list1)): # loop to iterate the review data
  if len(x_train_data_list1[i]) >= 10: # if the review length is bigger than 10
    x_train_data_list1[i] = x_train_data_list1[i][0:10] # select the first 10 word
    list2 = []
    for word in x_train_data_list1[i]: # loop to iterate each word
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word].tolist() # get the vector of word
        for item in vec:
          list2.append(item) # add the word vector to list
      else: # if word is not in vocablary
        for k in range(300):
          list2.append(0) # set the vector to 0 and add to list
  else: # if review length is less than 10
    list2 = []
    num = 10 - len(x_train_data_list1[i]) # count the number of we needed 
    for word in x_train_data_list1[i]: # loop to iterate each word
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word].tolist() # get the vector of word
        for item in vec:
          list2.append(item) # add the word vector to list
      else: # if word is not in vocablary
        for k in range(300):
          list2.append(0) # set the vector to 0
    for h in range(num):
      for o in range(300): # set word vector to 0 and add it to list
        list2.append(0)
  x_train_10vector_list.append(list2)

In [ ]:
x_train_10vector_list = np.array(x_train_10vector_list) # convert list to array
x_train_10vector_list.shape

(80000, 3000)

In [ ]:
x_test_10vector_list = [] # create a list to keep the first 10 word of each review of test data
x_test_data_list1 = x_test_data_list.copy()
for i in range(len(x_test_data_list1)): # loop to iterate the review data
  if len(x_test_data_list1[i]) >= 10: # if the review length is bigger than 10
    x_test_data_list1[i] = x_test_data_list1[i][0:10] # select the first 10 word
    list2 = []
    for word in x_test_data_list1[i]: # loop to iterate each word
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word].tolist() # get the vector of word
        for item in vec:
          list2.append(item) # add the word vector to list
      else: # if word is not in vocablary
        for k in range(300):
          list2.append(0) # set the vector to 0 and add to list
  else: # if review length is less than 10
    list2 = []
    num = 10 - len(x_test_data_list1[i]) # count the number of we needed 
    for word in x_test_data_list1[i]:  # loop to iterate each word
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word].tolist()  # get the vector of word
        for item in vec:
          list2.append(item) # add the word vector to list
      else: # if word is not in vocablary
        for k in range(300):
          list2.append(0) # set the vector to 0
    for h in range(num):
      for o in range(300): # set word vector to 0 and add it to list
        list2.append(0)
  x_test_10vector_list.append(list2)

In [ ]:
x_test_10vector_list = np.array(x_test_10vector_list) # convert list to list
x_test_10vector_list.shape

(20000, 3000)

In [ ]:
x_train_10vector_tensor = tf.convert_to_tensor(x_train_10vector_list) # convert train data to tensor type
x_test_10vector_tensor = tf.convert_to_tensor(x_test_10vector_list)# convert test data to tensor type

In [ ]:
MLP2 = Sequential()
MLP2.add(Dense(3000, input_shape=(3000,), activation='relu')) # input layer with 3000 node 
MLP2.add(Dense(50, activation='relu')) # first hidden layer with 50 node 
MLP2.add(Dense(10, activation='relu')) # second hidden layer with 10 node
MLP2.add(Dense(5, activation='sigmoid')) # output layer with 5 node beacuse there are 5 class

In [ ]:
MLP2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) # compile model
MLP2.fit(x_train_10vector_tensor, OHE_y_train_data, epochs=50, batch_size=500) # train the model

Epoch 1/50
160/160 [==============================] - 2s 9ms/step - loss: 1.3881 - accuracy: 0.3713
Epoch 2/50
160/160 [==============================] - 1s 9ms/step - loss: 1.2516 - accuracy: 0.4477
Epoch 3/50
160/160 [==============================] - 1s 9ms/step - loss: 1.1284 - accuracy: 0.5143
Epoch 4/50
160/160 [==============================] - 1s 9ms/step - loss: 0.9537 - accuracy: 0.6002
Epoch 5/50
160/160 [==============================] - 2s 11ms/step - loss: 0.7408 - accuracy: 0.7034
Epoch 6/50
160/160 [==============================] - 2s 11ms/step - loss: 0.5354 - accuracy: 0.7945
Epoch 7/50
160/160 [==============================] - 1s 9ms/step - loss: 0.3736 - accuracy: 0.8616
Epoch 8/50
160/160 [==============================] - 1s 9ms/step - loss: 0.2684 - accuracy: 0.9018
Epoch 9/50
160/160 [==============================] - 1s 9ms/step - loss: 0.2000 - accuracy: 0.9280
Epoch 10/50
160/160 [==============================] - 1s 9ms/step - loss: 0.1685 - accuracy: 0.93

In [ ]:
y_pre_10vector = MLP2.predict(x_test_10vector_tensor) # predict the y test
y_pre_10vector_list = []
for item in y_pre_10vector:
  y_pre_10vector_list.append(np.argmax(item,axis=0)+1) # select the class 

accuracy_10vector = accuracy_score(y_test_data, y_pre_10vector_list) # count the accuracy
accuracy_10vector

0.4027

The accuracy of first model is 0.33795, and accuracy of the second model is 0.4023, which is much better than first model. Since the number of data is same, the model is same, and the hyperparameter is same for both model, but the input vector of second model 3000, and the first model only have 300. In this case, i can conclude that the number of word vector have a big effect to the model. The more vector data have, the accury will be higher.

## 5. Recurrent Neural Networks

In [ ]:
x_train_20vector_list = [] # create a list to keep the first 20 word of each review of train data 
x_train_data_list2 = x_train_data_list.copy()
for i in range(len(x_train_data_list2)): # iterate the data list
  list1 = []
  if len(x_train_data_list2[i]) >= 20: # check the length of review data
    x_train_data_list2[i] = x_train_data_list2[i][0:20] # select first 20 word 
    for word in x_train_data_list2[i]:
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word] # get the word vector
        list1.append(vec) # add to list
      else: # if word is not in vocabulary
        list1.append(np.zeros(300)) # set 0 as vector and add to list
  else:
    num = 20 - len(x_train_data_list2[i]) # check the length 
    for word in x_train_data_list2[i]:
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word] # get the word vector
        list1.append(vec) # add to list
      else: # if word is not in vocabulary
        list1.append(np.zeros(300)) # set 0 as vector and add to list
    for n in range(num): # add the less word to list 
      list1.append(np.zeros(300))
  x_train_20vector_list.append(list1)


In [ ]:
x_train_20vector_list = np.array(x_train_20vector_list) # convert to array
x_train_20tensor = tf.convert_to_tensor(x_train_20vector_list) # convert the data type
x_train_20vector_list.shape   

(80000, 20, 300)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN

RNN_model = Sequential() 
RNN_model.add(SimpleRNN(300, input_shape=(None, 300))) # create the RNN model
RNN_model.add(Dense(20, activation='relu')) # hidden layer with 20 node
RNN_model.add(Dense(5, activation='sigmoid')) # output layer with 5 node


In [ ]:
from tensorflow import keras
RNN_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])# compile model

RNN_model.fit(x_train_20vector_list, OHE_y_train_data, epochs=50, batch_size=500) # train data

Epoch 1/50
160/160 [==============================] - 3s 13ms/step - loss: 1.3761 - accuracy: 0.3780
Epoch 2/50
160/160 [==============================] - 2s 14ms/step - loss: 1.2670 - accuracy: 0.4415
Epoch 3/50
160/160 [==============================] - 2s 14ms/step - loss: 1.2404 - accuracy: 0.4570
Epoch 4/50
160/160 [==============================] - 2s 13ms/step - loss: 1.2122 - accuracy: 0.4672
Epoch 5/50
160/160 [==============================] - 2s 13ms/step - loss: 1.1906 - accuracy: 0.4782
Epoch 6/50
160/160 [==============================] - 2s 14ms/step - loss: 1.1701 - accuracy: 0.4902
Epoch 7/50
160/160 [==============================] - 2s 12ms/step - loss: 1.1529 - accuracy: 0.4967
Epoch 8/50
160/160 [==============================] - 2s 12ms/step - loss: 1.1396 - accuracy: 0.5029
Epoch 9/50
160/160 [==============================] - 2s 12ms/step - loss: 1.1182 - accuracy: 0.5167
Epoch 10/50
160/160 [==============================] - 2s 14ms/step - loss: 1.1058 - accura

In [ ]:
x_test_20vector_list = [] # create a list to keep the first 20 word of each review of test data 
x_test_data_list2 = x_test_data_list.copy()
for i in range(len(x_test_data_list2)): # iterate the data list
  list1 = []
  if len(x_test_data_list2[i]) >= 20: # check the length of review data 
    x_test_data_list2[i] = x_test_data_list2[i][0:20] # select first 20 word
    for word in x_test_data_list2[i]:
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word] # get the wor vector
        list1.append(vec)
      else: # if word is not in vocabulary
        list1.append(np.zeros(300)) # set 0 as word vector
  else:
    num = 20 - len(x_test_data_list2[i]) # check the length 
    for word in x_test_data_list2[i]:
      if word in google_model.wv.vocab: # check if word is in vocabulary
        vec = google_model.wv[word] # get the wor vector
        list1.append(vec)
      else: # if word is not in vocabulary
        list1.append(np.zeros(300)) # set 0 as word vector
    for n in range(num):
      list1.append(np.zeros(300)) # add the less word vector
  x_test_20vector_list.append(list1)

In [ ]:
x_test_20tensor = tf.convert_to_tensor(x_test_20vector_list) # convert the data to tensor type 
y_pre_20vector = RNN_model.predict(x_test_20tensor) # predict the y test
y_pre_20vector_list = []
for item in y_pre_20vector:
  y_pre_20vector_list.append(np.argmax(item,axis=0)+1) # select the class result

accuracy_20vector = accuracy_score(y_test_data, y_pre_20vector_list) # get the accuracy
accuracy_20vector

0.42155

##(b)

In [ ]:
from tensorflow.keras.layers import GRU

RNN_model1 = Sequential() 
RNN_model1.add(GRU(300, input_shape=(None, 300))) # creat the GRU cell
RNN_model1.add(Dense(20, activation='relu')) # hidden layer with 20 node
RNN_model1.add(Dense(5, activation='sigmoid')) # output layer with 5 node

RNN_model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

RNN_model1.fit(x_train_20vector_list, OHE_y_train_data, epochs=50, batch_size=500) # train the model

Epoch 1/50
160/160 [==============================] - 6s 14ms/step - loss: 1.3492 - accuracy: 0.3904
Epoch 2/50
160/160 [==============================] - 2s 13ms/step - loss: 1.2265 - accuracy: 0.4595
Epoch 3/50
160/160 [==============================] - 2s 13ms/step - loss: 1.1802 - accuracy: 0.4812
Epoch 4/50
160/160 [==============================] - 2s 13ms/step - loss: 1.1563 - accuracy: 0.4905
Epoch 5/50
160/160 [==============================] - 2s 13ms/step - loss: 1.1279 - accuracy: 0.5045
Epoch 6/50
160/160 [==============================] - 2s 13ms/step - loss: 1.1079 - accuracy: 0.5113
Epoch 7/50
160/160 [==============================] - 2s 13ms/step - loss: 1.0903 - accuracy: 0.5207
Epoch 8/50
160/160 [==============================] - 2s 13ms/step - loss: 1.0744 - accuracy: 0.5271
Epoch 9/50
160/160 [==============================] - 2s 13ms/step - loss: 1.0542 - accuracy: 0.5375
Epoch 10/50
160/160 [==============================] - 2s 13ms/step - loss: 1.0315 - accura

In [ ]:
y_pre_20vector1 = RNN_model1.predict(x_test_20tensor) # predict the y test
y_pre_20vector_list1 = []
for item in y_pre_20vector1:
  y_pre_20vector_list1.append(np.argmax(item,axis=0)+1) # select the result

accuracy_20vector1 = accuracy_score(y_test_data, y_pre_20vector_list1) # get the accuracy
accuracy_20vector1

0.466

The accuracy of first model is 0.42155, and the accuracy of second model is 0.466, whcih is better than first model. Since number of input data and hyperparmeter are same for both model, we can say gated recurrent unit cell could make model better.